Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [2]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

#Feature Engineering 

In [3]:
#looking at the data 
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
#creating a list of the columns to pull from when creating features 
list(df.columns)

['bathrooms',
 'bedrooms',
 'created',
 'description',
 'display_address',
 'latitude',
 'longitude',
 'price',
 'street_address',
 'interest_level',
 'elevator',
 'cats_allowed',
 'hardwood_floors',
 'dogs_allowed',
 'doorman',
 'dishwasher',
 'no_fee',
 'laundry_in_building',
 'fitness_center',
 'pre-war',
 'laundry_in_unit',
 'roof_deck',
 'outdoor_space',
 'dining_room',
 'high_speed_internet',
 'balcony',
 'swimming_pool',
 'new_construction',
 'terrace',
 'exclusive',
 'loft',
 'garden_patio',
 'wheelchair_access',
 'common_outdoor_space']

In [5]:
#creating a feature for total number of perks the apartment offers
perks = [
 'elevator','cats_allowed','hardwood_floors','dogs_allowed',
 'doorman','dishwasher','no_fee','laundry_in_building','fitness_center',
 'pre-war','laundry_in_unit','roof_deck','outdoor_space','dining_room',
 'high_speed_internet','balcony','swimming_pool','new_construction',
 'terrace','exclusive','loft','garden_patio','wheelchair_access','common_outdoor_space'
 ]

df['total_perks'] = df[perks].sum(axis=1)

df[['total_perks', 'street_address']].head()

,total_perks,street_address
0,0,792 Metropolitan Avenue
1,5,808 Columbus Avenue
2,3,241 W 13 Street
3,2,333 East 49th Street
4,1,500 West 143rd Street


In [6]:
#creating a feature for total number of rooms in the apartment 
rooms = ['bedrooms', 'bathrooms']

df['total_rooms'] = df[rooms].sum(axis=1)

df[['bathrooms', 'bedrooms', 'total_rooms']].sample(5)

,bathrooms,bedrooms,total_rooms
13922,1.0,0,1.0
19028,1.0,3,4.0
47937,3.0,3,6.0
46394,2.0,3,5.0
293,1.0,0,1.0


In [18]:
#creating a column for month created to make splitting simpler
df['created'] = pd.to_datetime(df['created'], infer_datetime_format=True)

month = []
for date in df['created']:
  mon = date.month
  month.append(mon)

df['month_created'] = month

df[['created', 'month_created']].sample(5)

,created,month_created
2480,2016-04-10 03:39:47,4
47153,2016-04-15 02:43:40,4
13041,2016-06-02 06:07:00,6
21992,2016-06-14 17:43:24,6
43174,2016-04-20 01:22:24,4


In [21]:
#spliting the data into test and train sets 
train = df[df['month_created'] < 6]
test = df[df['month_created'] >= 6]

print(train.shape, test.shape)
train['month_created'].sample(20)

(31844, 37) (16973, 37)


8045     4
10502    4
6839     4
884      4
41764    4
42542    4
28105    5
2072     4
25632    5
158      4
27714    5
29422    5
31659    5
25709    5
39808    5
3249     4
29300    5
41894    4
42142    4
25599    5
Name: month_created, dtype: int64

In [23]:
test['month_created'].sample(20)

13375    6
25348    6
24113    6
14896    6
18212    6
20248    6
21874    6
11287    6
18396    6
3333     6
21636    6
29773    6
16105    6
12453    6
15313    6
23389    6
20113    6
20797    6
2731     6
165      6
Name: month_created, dtype: int64

#Running Linear Regression

##baseline

In [24]:
#getting the target data into proper format 
target = 'price'
y_train = train[target]
y_test = test[target]
y_train.shape, y_test.shape

((31844,), (16973,))

In [25]:
#making my guess the mean as a baseline 
guess = y_train.mean()
guess

3575.604007034292

In [29]:
#finding the baseline error on the training data 
from sklearn.metrics import mean_absolute_error
y_pred = [guess] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'Train error is : {mae:.2f} dollars')

Train error is : 1201.88 dollars


In [31]:
#finding the baseline error on the test data 
from sklearn.metrics import mean_absolute_error
y_pred = [guess] * len(y_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test error is : {mae:.2f} dollars')

Test error is : 1197.71 dollars


##With Features (attempt one)

In [35]:
#creating dataframes for features and target 
features = ['total_perks', 'total_rooms']
target = ['price']

x_train = train[features]
y_train = train[target]
x_test = test[features]
y_test = test[target]

In [36]:
#importing linear regression and instaniating the model
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [37]:
#fitting the model to the training data
model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [56]:
#testing the mean absolute error on the training data

y_pred_train = model.predict(x_train)
mae = mean_absolute_error(y_train, y_pred_train)
print(f'Train error: {mae:.2f} dollars')

Train error: 856.62 dollars


In [43]:
#testing the model on new data in test dataframe 

y_pred = model.predict(x_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test error: {mae:.2f} dollars')

Test error: 868.00 dollars


#finding the coefficent and intercept of the model

In [48]:
#finding coefficent and intercept 
model.intercept_, model.coef_

(array([998.19463411]), array([[101.76435155, 767.95662272]]))

In [52]:
#putting it into an easier to read format 
print('Intercept', model.intercept_)
print('Price per perk', model.coef_[0][0])
print('Price per room', model.coef_[0][1])

Intercept [998.19463411]
Price per perk 101.76435154698721
Price per room 767.956622719957


##Getting Regression Metrics 

In [54]:
#importing mean squared error and r2_score 
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt

In [57]:
#finding the root mean squared error (rmse) for test and train
trainmse = mean_squared_error(y_train, y_pred_train)
trainrsme = sqrt(trainmse)

testmse = mean_squared_error(y_test, y_pred)
testrsme = sqrt(testmse)

print('Train RSME is:', trainrsme)
print('Test RSME is:', testrsme)

Train RSME is: 1295.7149658677172
Test RSME is: 1293.2257015977382


In [59]:
#finding train and test r^2
trainr2 = r2_score(y_train, y_pred_train)
testr2 = r2_score(y_test, y_pred)

print('Train r^2:', trainr2)
print('Test r^2:', testr2)

Train r^2: 0.45930369872329946
Test r^2: 0.46189791795456303
